In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
import os
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Importación de los datos

In [3]:
# Leer el archivo CSV
df = pd.read_csv('diabetic_data.csv')

# Mostrar las primeras filas del dataset
print(df.head())

# Resumen de la información del dataset
print(df.info())

# Mostrar la cantidad de valores nulos por columna
print(df.isnull().sum())

   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No

In [4]:
# Leer el archivo CSV, tratando '?' como valores nulos
df = pd.read_csv('diabetic_data.csv', na_values='?')

# Mostrar la cantidad de valores nulos por columna
print(df.isnull().sum())

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv('diabetic_data_modified.csv', index=False)

print("El archivo modificado ha sido guardado como 'diabetic_data_modified.csv'.")


C:\Users\grego\AppData\Local\Temp\ipykernel_9000\4281049621.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('diabetic_data.csv', na_values='?')


encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

 ### Procesamiento de los datos

In [5]:
# Cargar el dataset
df = pd.read_csv('diabetic_data_modified.csv')

# Asegurarse de manejar correctamente los valores nulos
# Identificar las columnas con valores nulos según la descripción proporcionada
cols_with_missing = ['race', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3']

# Imputar los valores nulos con la moda para variables categóricas
categorical_imputer = SimpleImputer(strategy='most_frequent')
df[cols_with_missing] = categorical_imputer.fit_transform(df[cols_with_missing])

# Limpiar y convertir 'weight' a valores numéricos y luego categorizarlo
def clean_weight(weight_str):
    if weight_str == "?":
        return None
    elif weight_str.startswith(">"):
        return float(weight_str[1:]) + 1  # Incrementar en 1 para asegurar que los límites sean correctos
    elif weight_str.startswith("["):
        return float(weight_str.strip("[]").split("-")[0])
    elif weight_str == "Unknown":
        return None
    else:
        return float(weight_str)

df['weight'] = df['weight'].apply(clean_weight)

# Definir los rangos de peso y codificar 'weight'
weight_ranges = ['[0-25)', '[25-50)', '[50-75)', '[75-100)', '[100-125)', '[125-150)', '[150-175)', '[175-200)', '>200']
df['weight_category'] = pd.cut(df['weight'], bins=[0, 25, 50, 75, 100, 125, 150, 175, 200, float('inf')], labels=weight_ranges, right=False)

# Eliminar la columna original 'weight'
df.drop(columns=['weight'], inplace=True)

# Función para asignar categorías a los códigos ICD-9
def assign_icd_category(icd_code):
    if pd.isnull(icd_code):
        return 'Unknown'
    if icd_code.startswith(('E', 'V')):
        return 'E-V codes'
    else:
        code_number = int(icd_code.split('.')[0])  # Tomar solo el número de código ICD-9
        if 1 <= code_number <= 139:
            return '001-139'
        elif 140 <= code_number <= 239:
            return '140-239'
        elif 240 <= code_number <= 279:
            return '240-279'
        elif 280 <= code_number <= 289:
            return '280-289'
        elif 290 <= code_number <= 319:
            return '290-319'
        elif 320 <= code_number <= 389:
            return '320-389'
        elif 390 <= code_number <= 459:
            return '390-459'
        elif 460 <= code_number <= 519:
            return '460-519'
        elif 520 <= code_number <= 579:
            return '520-579'
        elif 580 <= code_number <= 629:
            return '580-629'
        elif 630 <= code_number <= 679:
            return '630-679'
        elif 680 <= code_number <= 709:
            return '680-709'
        elif 710 <= code_number <= 739:
            return '710-739'
        elif 740 <= code_number <= 759:
            return '740-759'
        elif 760 <= code_number <= 779:
            return '760-779'
        elif 780 <= code_number <= 799:
            return '780-799'
        elif 800 <= code_number <= 999:
            return '800-999'
        else:
            return 'Other'  # En caso de no encontrar una categoría válida

# Aplicar la función a cada columna de diagnóstico
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[col + '_category'] = df[col].apply(assign_icd_category)

# Eliminar las columnas originales de diagnóstico
df.drop(columns=['diag_1', 'diag_2', 'diag_3'], inplace=True)

# Función para asignar valores únicos a las franjas de edad
def age_to_value(age_str):
    age_mapping = {
        '[0-10)': 5,
        '[10-20)': 15,
        '[20-30)': 25,
        '[30-40)': 35,
        '[40-50)': 45,
        '[50-60)': 55,
        '[60-70)': 65,
        '[70-80)': 75,
        '[80-90)': 85,
        '[90-100)': 95
    }
    return age_mapping.get(age_str, None)

# Aplicar la función de agrupamiento de edades
df['age'] = df['age'].apply(age_to_value)

# Aplicar codificación one-hot a las variables categóricas, excluyendo 'age' ya que está mapeada a valores únicos
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if 'age' in categorical_cols:
    categorical_cols.remove('age')
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_cols]))

# Sustituir las columnas originales con las nuevas codificadas
encoded_cols.columns = encoder.get_feature_names_out(categorical_cols)
df.drop(columns=categorical_cols, inplace=True)
df = pd.concat([df, encoded_cols], axis=1)

# Guardar el resultado en un nuevo archivo CSV
# df.to_csv('processed_data.csv', index=False)

print("Datos procesados y guardados en 'processed_data.csv'.")

C:\Users\grego\AppData\Local\Temp\ipykernel_9000\2766882862.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('diabetic_data_modified.csv')


Datos procesados y guardados en 'processed_data.csv'.


Para implementar un árbol de decisión en PyTorch, tendrás que crear una red neuronal que emule el comportamiento de un árbol de decisión, ya que PyTorch está orientado a redes neuronales y no proporciona una implementación directa de árboles de decisión. Sin embargo, se puede simular el comportamiento de un árbol de decisión mediante una red neuronal con capas especializadas que emulen las decisiones binarias

### Creación del modelo de árbol de decisión con Pytorch

In [33]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Función para cargar los datos
def load_data(filepath):
    return pd.read_csv(filepath)

# Función para dividir los datos en partes para simular diferentes dispositivos
def split_data(X, y, num_parts):
    X_splits = np.array_split(X, num_parts)
    y_splits = np.array_split(y, num_parts)
    return X_splits, y_splits

# Definir la red neuronal que simula un árbol de decisión
class DecisionTreeNN(nn.Module):
    def __init__(self, input_size):
        super(DecisionTreeNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.5)
        self.fc5 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc5(x))
        return x

# Función para entrenar el modelo en un dispositivo
def train_on_device(X_train, y_train, X_val, y_val, model, epochs=10, lr=0.0001, weight_decay=1e-4, batch_size=64):
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
    
    dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        
        # Evaluar el modelo local en datos de validación del dispositivo
        model.eval()
        with torch.no_grad():
            outputs = model(X_val_tensor)
            predicted = (outputs.numpy() > 0.5).astype(int)
            val_accuracy = accuracy_score(y_val_tensor.numpy(), predicted)
            print(f'Epoch {epoch + 1}/{epochs}, Accuracy del modelo local en validación: {val_accuracy:.5f}')
    
    return model, val_accuracy

# Función principal para la simulación de Federated Learning
def federated_learning(X, y, num_parts, epochs=10, lr=0.0001, weight_decay=1e-4, batch_size=64):
    X_splits, y_splits = split_data(X, y, num_parts)
    local_models = []
    scalers = []
    local_accuracies = []
    global_accuracy = 0.0

    for i in range(num_parts):
        X_train, X_test, y_train, y_test = train_test_split(X_splits[i], y_splits[i], test_size=0.4, stratify=y_splits[i])
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)  # Dividir datos de entrenamiento en entrenamiento y validación
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        X_test_scaled = scaler.transform(X_test)
        
        local_model = DecisionTreeNN(X_train_scaled.shape[1])
        local_model, val_accuracy = train_on_device(X_train_scaled, y_train, X_val_scaled, y_val, local_model, epochs=epochs, lr=lr, weight_decay=weight_decay, batch_size=batch_size)
        local_models.append(local_model)
        scalers.append(scaler)
        
        # Evaluar el modelo local en datos de prueba del dispositivo
        X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
        
        local_model.eval()
        with torch.no_grad():
            outputs = local_model(X_test_tensor)
            predicted = (outputs.numpy() > 0.5).astype(int)
            local_accuracy = accuracy_score(y_test_tensor.numpy(), predicted)
            local_accuracies.append(local_accuracy)
            print(f'Accuracy del modelo local en el dispositivo {i + 1}: {local_accuracy:.5f}')
            print()
    
    # Calcular el accuracy global como el promedio de los accuracies locales
    global_accuracy = sum(local_accuracies) / num_parts
    
    # Evaluar el modelo global en datos combinados de prueba
    X_combined_scaled = np.vstack([scalers[i].transform(X_splits[i]) for i in range(num_parts)])
    y_combined = np.hstack([y_splits[i] for i in range(num_parts)])
    
    X_combined_tensor = torch.tensor(X_combined_scaled, dtype=torch.float32)
    y_combined_tensor = torch.tensor(y_combined, dtype=torch.float32).view(-1, 1)
    
    global_model = DecisionTreeNN(X.shape[1])
    global_state_dict = global_model.state_dict()
    
    for key in global_state_dict.keys():
        global_state_dict[key] = torch.mean(torch.stack([model.state_dict()[key].float() for model in local_models]), dim=0)
    
    global_model.load_state_dict(global_state_dict)
    
    global_model.eval()
    with torch.no_grad():
        outputs = global_model(X_combined_tensor)
        predicted = (outputs.numpy() > 0.5).astype(int)
        global_accuracy_combined = accuracy_score(y_combined_tensor.numpy(), predicted)
        print(f'Accuracy del modelo global en datos combinados de prueba: {global_accuracy_combined:.5f}')
    
    return global_accuracy, global_accuracy_combined

# Número de partes para simular diferentes dispositivos
num_parts = 5

# Cargar datos desde el archivo CSV
df = load_data('processed_data.csv')

# Seleccionar características y etiquetas
X = df.drop(columns=['diabetesMed_Yes'])
y = df['diabetesMed_Yes']

# Ejecutar una sola iteración de federated learning
global_accuracy, global_accuracy_combined = federated_learning(X, y, num_parts)

# Imprimir el accuracy global después de una iteración
print(f'Accuracy del modelo global después de una iteración: {global_accuracy:.5f}')

# Imprimir el accuracy del modelo global en datos combinados de prueba
print(f'Accuracy del modelo global en datos combinados de prueba: {global_accuracy_combined:.5f}')


Epoch 1/10, Accuracy del modelo local en validación: 0.73230
Epoch 2/10, Accuracy del modelo local en validación: 0.98608
Epoch 3/10, Accuracy del modelo local en validación: 0.99918
Epoch 4/10, Accuracy del modelo local en validación: 0.99959
Epoch 5/10, Accuracy del modelo local en validación: 0.99959
Epoch 6/10, Accuracy del modelo local en validación: 0.99959
Epoch 7/10, Accuracy del modelo local en validación: 1.00000
Epoch 8/10, Accuracy del modelo local en validación: 1.00000
Epoch 9/10, Accuracy del modelo local en validación: 1.00000
Epoch 10/10, Accuracy del modelo local en validación: 1.00000
Accuracy del modelo local en el dispositivo 1: 0.99889

Epoch 1/10, Accuracy del modelo local en validación: 0.75808
Epoch 2/10, Accuracy del modelo local en validación: 0.94228
Epoch 3/10, Accuracy del modelo local en validación: 0.99386
Epoch 4/10, Accuracy del modelo local en validación: 0.99673
Epoch 5/10, Accuracy del modelo local en validación: 0.99754
Epoch 6/10, Accuracy del mod